# 06. 토픽 모델링(Topic Modeling)

## 6.1. 잠재 의미 분석(Latent Semantic Analysis, LSA)

LSA...는 LDA의 전 단계면서 토픽 모델링에 기여한 모델입니다.

BoW에 기반한 DTM이나 TF-IDF는 단어의 빈도 수를 이용한 수치화 기법이기 때문에 단어나 문장의 의미를 고려하지 못한다는 문제점이 있습니다. 토픽을 고려하지 못한다고 하지요.

그래서 나온 것이 DTM의 잠재된(Latent) 의미를 이끌어내는 방법으로 LSA를 쓰게 되었습니다. LSA를 이해하기 위해서는 선형대수의 특이값 분해(SVD)를 이해해야 합니다.

### 6.1.1. 특이값 분해(Singular Value Decomposition, SVD)

SVD란 A가 m x n 행렬일 때, 다음과 같이 3개의 행렬의 곱으로 분해 되는 것을 말합니다.

A=UΣVT

여기서 각 3개의 행렬은
- U:m×m 직교행렬 (AAT=U(ΣΣT)UT)
- V:n×n 직교행렬 (ATA=V(ΣTΣ)VT)
- Σ:m×n 직사각 대각행렬

입니다.

간단히 얘기하겠습니다.

1. 전치행렬 : 행렬의 주대각선을 축으로 대칭으로 뒤집은 행렬입니다.
2. 단위행렬 : 주대각선이 0이고, 나머지 원소가 모두 0인 행렬입니다.
3. 역행렬 : 어떤 행렬 A에 행렬 B를 곱했을 때 단위행렬이 나온다면, B는 A의 역행렬이다.
4. 직교행렬 : 어떤 행렬 A에 A의 전치행렬을 곱했을 때, 단위행렬이 나온다면, 행렬 A는 직교행렬이다.
5. 대각행렬 : 주대각선을 제외한 모든 원소가 0인 행렬이다.

### 6.1.2. 절단된 SVD

그런데 위에서 설명한 SVD는 풀SVD라고 하며, LSA의 경우는 풀SVD가 아닌 절단된 SVD를 사용합니다.

절단된 SVD는 대각행렬 시그마의 대각원소의 값 중에서 t개만 남깁니다. 물론 기존 행렬 A를 복구할 수 없습니다.

U행렬과 V행렬의 t열까지만 남깁니다. 여기서 t는 우리가 찾고자하는 토픽의 수를 반영한 하이퍼파라미터값입니다.

t를 크게 잡으면 다양한 의미를 가져갈 수 있지만, t를 작게 잡으면 노이즈를 제거할 수 있습니다.

### 6.1.3. 실습

In [1]:
# 문서 4개의 DTM을 만들어 봅니다.

import numpy as np
A=np.array([[0,0,0,1,0,1,1,0,0],
            [0,0,0,1,1,0,1,0,0],
            [0,1,1,0,2,0,0,0,0],
            [1,0,0,0,0,0,0,1,1]])
np.shape(A)

(4, 9)

In [2]:
# 풀SVD를 수행해보겠습니다. 대각행렬은 S를 사용합니다.

U, S, VT = np.linalg.svd(A, full_matrices=True)

In [3]:
# A는 4 X 9 의 행렬입니다.
# U는 4 X 4 의 직교행렬입니다.

print(U.round(2))
np.shape(U)

[[ 0.24  0.75  0.    0.62]
 [ 0.51  0.44 -0.   -0.74]
 [ 0.83 -0.49 -0.    0.27]
 [ 0.   -0.    1.   -0.  ]]


(4, 4)

In [4]:
# S는 대각행렬입니다. 그러나 Numpy의 linalg.svd()는 리스트로 반환합니다.

print(S.round(2))
np.shape(S)

[2.69 2.05 1.73 0.77]


(4,)

In [5]:
# 대각행렬로 보려면 아래와 같은 방식을 취해야 합니다.

S_ = np.zeros((4, 9)) # 4 X 9의 영행렬을 만듭니다.
S_[:4, :4] = np.diag(S) # 특이값을 대각행렬에 삽입합니다.
print(S_.round(2))
np.shape(S_)

[[2.69 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   2.05 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   1.73 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.77 0.   0.   0.   0.   0.  ]]


(4, 9)

In [7]:
# VT는 9 X 9 의 직교행렬입니다. V의 전치행렬입니다.

print(VT.round(2))
np.shape(VT)

[[ 0.    0.31  0.31  0.28  0.8   0.09  0.28  0.    0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]
 [ 0.58 -0.    0.    0.   -0.    0.   -0.    0.58  0.58]
 [-0.    0.35  0.35 -0.16 -0.25  0.8  -0.16  0.    0.  ]
 [-0.   -0.78 -0.01 -0.2   0.4   0.4  -0.2   0.    0.  ]
 [-0.29  0.31 -0.78 -0.24  0.23  0.23  0.01  0.14  0.14]
 [-0.29 -0.1   0.26 -0.59 -0.08 -0.08  0.66  0.14  0.14]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19  0.75 -0.25]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19 -0.25  0.75]]


(9, 9)

In [9]:
# U X S X VT = A 가 나와야 합니다.

np.allclose(A, np.dot(np.dot(U, S_), VT).round(2))

True

In [10]:
# 이제 t를 정하고 절단된 SVD를 수행해보도록 하겠습니다. t=2로 하겠습니다.

S = S_[:2, :2] # S_ 대각행렬의 상위 2개만 남기고 제거
print(S.round(2))

[[2.69 0.  ]
 [0.   2.05]]


In [11]:
# U도 상위 2개만 남기고 제거

U = U[:, :2]
print(U.round(2))

[[ 0.24  0.75]
 [ 0.51  0.44]
 [ 0.83 -0.49]
 [ 0.   -0.  ]]


In [12]:
# VT도 마찬가지

VT = VT[:2, :]
print(VT.round(2))

[[ 0.    0.31  0.31  0.28  0.8   0.09  0.28  0.    0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]]


In [15]:
# 이제 축소된 U, S, VT 에 대해서 연산을 하면 A와는 다른 값이 나올 겁니다.
# 이 값을 A_ 이라고 하고 A와 비교해보겠습니다.

A_ = np.dot(np.dot(U, S), VT)
print(A)
print(A_.round(2))

[[0 0 0 1 0 1 1 0 0]
 [0 0 0 1 1 0 1 0 0]
 [0 1 1 0 2 0 0 0 0]
 [1 0 0 0 0 0 0 1 1]]
[[ 0.   -0.17 -0.17  1.08  0.12  0.62  1.08 -0.   -0.  ]
 [ 0.    0.2   0.2   0.91  0.86  0.45  0.91  0.    0.  ]
 [ 0.    0.93  0.93  0.03  2.05 -0.17  0.03  0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.  ]]


대체적으로 기존에 0인 값들은 0에 가까운 값이 나오고, 1인 값들은 1에 가까운 값이 나옵니다.

축소된 U는 4 X 2 의 크기를 가지는데, 이는 (문서의 개수 X 토픽의 수t) 의 크기입니다.

마찬가지로 VT도 9 X 9 에서 2 X 9가 되었으니 이는 (토픽의 수 X 단어의 개수의 크기) 입니다.

### 6.1.4. 레알 실습

사이킷런에서 Twenty Newsgroups 라고 불리는 20개의 다른 주제를 가진 뉴스그룹 데이터를 제공합니다.

In [1]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups

In [2]:
data = fetch_20newsgroups(shuffle=True, random_state=42, remove=('headers', 'footers', 'quotes'))
documents = data.data
len(documents)

11314

In [3]:
documents[1]

"A fair number of brave souls who upgraded their SI clock oscillator have\nshared their experiences for this poll. Please send a brief message detailing\nyour experiences with the procedure. Top speed attained, CPU rated speed,\nadd on cards and adapters, heat sinks, hour of usage per day, floppy disk\nfunctionality with 800 and 1.4 m floppies are especially requested.\n\nI will be summarizing in the next two days, so please add to the network\nknowledge base if you have done the clock upgrade and haven't answered this\npoll. Thanks."

뉴스그룹 데이터에는 특수문자가 포함된 다수의 영어문장으로 구성되어 있습니다. 샘플은 11,314개이며 target_name에는 본래 이 뉴스그룹 데이터가 어떤 20개의 카테고리를 갖고 있었는지가 저장되어 있습니다.

In [5]:
print(data.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


#### 6.1.4.2 전처리

In [6]:
news_df = pd.DataFrame({'document': documents})

In [7]:
# 정규식을 통한 특수문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")

In [8]:
# 길이가 3이하인 단어는 제거
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3]))

In [9]:
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

In [10]:
news_df['clean_doc'][1]

'fair number brave souls upgraded their clock oscillator have shared their experiences this poll please send brief message detailing your experiences with procedure speed attained rated speed cards adapters heat sinks hour usage floppy disk functionality with floppies especially requested will summarizing next days please network knowledge base have done clock upgrade haven answered this poll thanks'

In [11]:
# 이제 토큰화를 시키고, 불용어를 처리합니다.

# NLTK에서 불용어 사전을 받습니다.
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [12]:
# 토큰화 합니다.
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split())

In [13]:
# 불용어 처리합니다.
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

In [14]:
print(tokenized_doc[1])

['fair', 'number', 'brave', 'souls', 'upgraded', 'clock', 'oscillator', 'shared', 'experiences', 'poll', 'please', 'send', 'brief', 'message', 'detailing', 'experiences', 'procedure', 'speed', 'attained', 'rated', 'speed', 'cards', 'adapters', 'heat', 'sinks', 'hour', 'usage', 'floppy', 'disk', 'functionality', 'floppies', 'especially', 'requested', 'summarizing', 'next', 'days', 'please', 'network', 'knowledge', 'base', 'done', 'clock', 'upgrade', 'answered', 'poll', 'thanks']


#### 6.1.4.3 TF-IDF 행렬 만들기

불용어 제거를 위해 토큰화 작업을 수행했지만, TfidfVectorizer는 기본적으로 토큰화가 되어있지 않은 텍스트 데이터를 입력 받스빈다. 따라서 역토큰화를 수행하겠습니다.

In [15]:
# 역토큰화 실시!
detokenized_doc = []
for i in range(len(news_df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)
    
news_df['clean_doc'] = detokenized_doc

In [16]:
news_df['clean_doc'][1]

'fair number brave souls upgraded clock oscillator shared experiences poll please send brief message detailing experiences procedure speed attained rated speed cards adapters heat sinks hour usage floppy disk functionality floppies especially requested summarizing next days please network knowledge base done clock upgrade answered poll thanks'

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
vectorizer = TfidfVectorizer(stop_words='english',
                            max_features=1000, # 상위 1000개의 단어를 보존
                            max_df = 0.5,
                            smooth_idf=True)

X = vectorizer.fit_transform(news_df['clean_doc'])
X.shape

(11314, 1000)

#### 6.1.4.4. 토픽 모델링(Topic Modeling)

이제 TF-IDF 행렬을 다수의 행렬로 분해해보도록 하겠습니다. 사이킷런의 Truncated SVD를 사용합니다.

원래 기존 뉴스그룹 데이터가 20개의 카테고리를 가지고 있었기 때문에, 20개의 토픽을 가졌다고 가정하고 토픽 모델링을 시도하겠습니다. n_components의 파리미터로 지정 가능합니다.

In [38]:
from sklearn.decomposition import TruncatedSVD

In [39]:
svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=200, random_state=42)
svd_model.fit(X)
len(svd_model.components_)

20

In [40]:
# 여기서 svd_model.components_ 는 앞서 배운 LSA에서 VT에 해당됩니다.

np.shape(svd_model.components_)

(20, 1000)

In [41]:
# 정확하게 토픽의 개수, 단어의 개수입니다.

terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d: " % (idx+1),
              [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n -1:-1]])
        
get_topics(svd_model.components_, terms)

Topic 1:  [('like', 0.21386), ('know', 0.20035), ('people', 0.19307), ('think', 0.178), ('good', 0.1513)]
Topic 2:  [('thanks', 0.32915), ('windows', 0.29086), ('card', 0.18016), ('drive', 0.1743), ('mail', 0.1513)]
Topic 3:  [('game', 0.37139), ('team', 0.32489), ('year', 0.28171), ('games', 0.25396), ('season', 0.18455)]
Topic 4:  [('drive', 0.53067), ('scsi', 0.20099), ('hard', 0.15546), ('disk', 0.15534), ('card', 0.14024)]
Topic 5:  [('windows', 0.40469), ('file', 0.25534), ('window', 0.18034), ('files', 0.16132), ('program', 0.13949)]
Topic 6:  [('chip', 0.1604), ('government', 0.15995), ('mail', 0.15641), ('space', 0.15112), ('information', 0.13587)]
Topic 7:  [('like', 0.67312), ('bike', 0.14188), ('chip', 0.1111), ('know', 0.10726), ('sounds', 0.10386)]
Topic 8:  [('card', 0.45931), ('video', 0.2186), ('sale', 0.21398), ('monitor', 0.15227), ('offer', 0.14739)]
Topic 9:  [('know', 0.45849), ('card', 0.34347), ('chip', 0.17473), ('government', 0.15081), ('video', 0.14754)]
Topi

## 6.2. 잠재 디리클레 할당(Latent Dirichlet Allocation, LDA)

토픽 모델링은 문서의 집합에서 토픽을 찾아내는 프로세스를 말하는데, 이는 검색 엔진, 고객 민원 시스템 등과 같이 문서의 주제를 알아내는 일이 중요한 곳에서 사용됩니다.

LDA는 문서들은 토픽들의 혼합으로 구성되어져 있으며, 토픽들은 확률분포에 기반하여 단어들을 생성한다고 가정! LDA는 문서가 생성되던 과정을 역추적한다.

### 6.2.2. LDA의 가정

LDA는 단어의 순서는 신경쓰지 않습니다.

문서 작성자는 이런 생각을 합니다. 나는 이 문서를 작성하기 위해서 이런 주제들을 넣을거고, 이런 주제들을 위해서는 이런 단어들을 넣을 거야!

1. 문서에 사용할 단어의 개수 N을 정합니다.
2. 문서에 사용할 토픽의 혼합을 확률 분포에 기반하여 결정합니다.
3. 문서에 사용할 각 단어를 정합니다.
4. 토픽 분포에서 토픽 T를 확률적으로 고릅니다.
5. 선택한 토픽 T에서 단어의 출현 확률 분포에 기반해 문서에 사용할 단어를 고릅니다.

이러한 과정을 역추적합니다.

### 6.2.3. LDA 수행하기

1. 사용자는 알고리즘에게 토픽의 개수 k를 알려줍니다.
2. 모든 단어를 k개 중 하나의 토픽에 할당합니다.
3. 이제 모든 문서의 모든 단어에 대해서 아래의 사항을 반복 진행합니다.

- 어떤 문서의 각 단어 w는 자신은 잘못된 토픽에 할당되어져 있지만, 다른 단어들은 전부 올바른 토픽에 할당되어져 있는 상태라고 가정합니다. 이에 따라 단어 w는 두 가지 기준에 따라서 토픽이 재할당 됩니다.
    - p(topic t | document d): 문서 d의 단어들 중 토픽 t에 해당하는 단어들의 비율; 즉 단어 w의 토픽을 할당하기 위해서 단어 w가 속해 있는 문서 d에 속한 단어들이 어떤 토픽에 속해있는지를 보고 결정한다는 것.
    - p(word w | topic t): 단어 w를 갖고 있는 모든 문서들 중 토픽 t가 할당된 비율; 즉 단어 w의 토픽을 할당하기 위해서 모든 문서 d(i)에서 단어 w가 어떤 토픽 t(j)에 속해 있는지를 보고 결정한다는 것.

### 6.2.4. LSA & LDA

- LSA : DTM을 차원 축소하여 축소 차원에서 근접 단어들을 토픽으로 묶는다.
- LDA : 단어가 특정 토픽에 존재할 확률과 문서에 특정 토픽이 존재할 확률을 결합확률로 추정하여 토픽을 추출한다.

### 6.2.5. 실습 with gensim

#### 6.2.5.1. 정수 인코딩과 단어 집합 만들기

In [19]:
tokenized_doc[:5]

0    [wondering, anyone, could, enlighten, door, sp...
1    [fair, number, brave, souls, upgraded, clock, ...
2    [well, folks, plus, finally, gave, ghost, week...
3    [weitek, address, phone, number, like, informa...
4    [article, owcb, world, tombaker, world, baker,...
Name: clean_doc, dtype: object

이제 각 단어에 정수 인코딩을 하는 동시에, 각 뉴스에서의 단어의 빈도수를 기록해 보겠습니다. word_id는 단어가 정수 인코딩된 값이고, word_frequency는 해당 뉴스에서의 해당 단어의 빈도수를 의미합니다. 이는 gensim의 corpora.Dictionary() 를 사용합니다.

In [24]:
from gensim import corpora

In [25]:
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두 번째 뉴스 출력. 첫 번째 문서의 인덱스는 0

[(24, 2), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 2), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 2), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 2), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 2), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1)]


In [27]:
print(dictionary[24])

please


In [28]:
len(dictionary)

64281

#### 6.2.5.2. LDA 모델 훈련시키기

기존의 뉴스 데이터가 총 20개의 카테고리를 가지고 있었으므로 토픽의 개수를 20으로 하여 LDA모델을 학습시켜보겠습니다.

In [32]:
import gensim
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

In [30]:
NUM_TOPICS = 20 # k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, # 정수로 인코딩된 문서를 받는다.
                                           num_topics=NUM_TOPICS, # 토픽의 개수를 받는다.
                                           id2word=dictionary, # 단어를 받는다.
                                           passes=15)# passes 는 신경망의 epochs 라고 보면 된다.
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.017*"drive" + 0.013*"card" + 0.012*"thanks" + 0.011*"system"')
(1, '0.015*"like" + 0.014*"would" + 0.010*"good" + 0.010*"time"')
(2, '0.021*"team" + 0.020*"game" + 0.013*"games" + 0.012*"play"')
(3, '0.014*"armenian" + 0.014*"said" + 0.012*"armenians" + 0.012*"turkish"')
(4, '0.010*"wire" + 0.009*"ground" + 0.009*"firearm" + 0.009*"picture"')
(5, '0.011*"people" + 0.005*"world" + 0.005*"children" + 0.005*"government"')
(6, '0.013*"government" + 0.011*"encryption" + 0.010*"public" + 0.010*"security"')
(7, '0.043*"space" + 0.017*"nasa" + 0.011*"earth" + 0.011*"university"')
(8, '0.012*"power" + 0.009*"water" + 0.007*"high" + 0.007*"model"')
(9, '0.016*"president" + 0.007*"states" + 0.007*"health" + 0.007*"bill"')
(10, '0.008*"baltimore" + 0.008*"nrhj" + 0.007*"pens" + 0.006*"espn"')
(11, '0.021*"medical" + 0.018*"disease" + 0.018*"food" + 0.017*"patients"')
(12, '0.016*"file" + 0.011*"program" + 0.008*"files" + 0.008*"window"')
(13, '0.013*"mail" + 0.011*"list" + 0.010*"informatio

- topic : 당신이 가설로 잡은 토픽의 갯수는?
- chunksize : 얼마나 많은 문서가 훈련 알고리즘에 사용되는가? 만약에 빠른 학습이 중요하시다면, 청크사이즈를 키워서 돌려봅시다! Hoffman의 논문에 의하면 Chunksize는 모델 품질에 영향을 미치지만 차이그 그렇게 크진 않다고 합니다!
- passes : 패스는 모델 학습시 전체 코퍼스에서 모델을 학습시키는 빈도를 제어한다고 합니다. epochs 와 같은 용어 같다!
- iteration : 각각 문서에 대해서 루프를 얼마나 돌리는지를 제어한다고 합니다. pass & iteration 은 최대한 많은게 좋다!
- eval_every = 1 in LdaModel
- alpha, eta = auto, 디리클레 분포의 감마함수에 대한 파라미터입니다!

#### 여기서 잠깐 LDA의 평가 기준

1. Perplexity : 혼란도. 값이 작을수록 토픽모델이 문서에 잘 반영된다고 볼 수 있다.
2. Coherence : 주제의 일관성. 한 문서 안에 유사한 단어가 많이 모여있는 정도를 표현한다.

In [34]:
cm = CoherenceModel(model=ldamodel, corpus=corpus, coherence='u_mass')
coherence = cm.get_coherence()
print("Cpherence",coherence)
print('\nPerplexity: ', ldamodel.log_perplexity(corpus))

Cpherence -4.769431264358465

Perplexity:  -10.676955554539967


다시 분석으로 돌아가서

#### 6.2.5.3. LDA시각화 하기

In [37]:
import pyLDAvis.gensim

In [38]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

#### 6.2.5.4. 문서 별 토픽 분포 보기

위에서 토픽 별 단어 분포는 확인하였으나, 아직 문서 별 토픽 분포에 대해선 확인하지 못했습니다.

각 문서의 토픽 분포는 이미 훈련된 LDA 모델에서 전체 데이터가 정수 인코딩 된 결과를 넣은 후에 확인이 가능합니다.

In [39]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i ==5:
        break
    print(i, ' 번째 문서의 topic 비율은', topic_list)

0  번째 문서의 topic 비율은 [(0, 0.1286175), (1, 0.7059573), (16, 0.14229512)]
1  번째 문서의 topic 비율은 [(0, 0.2841038), (1, 0.39486736), (12, 0.1759603), (13, 0.04642022), (16, 0.082651585)]
2  번째 문서의 topic 비율은 [(0, 0.29667747), (1, 0.48274583), (2, 0.036661465), (4, 0.020794887), (10, 0.03121134), (16, 0.12628786)]
3  번째 문서의 topic 비율은 [(6, 0.39452332), (12, 0.49292785)]
4  번째 문서의 topic 비율은 [(7, 0.04274679), (12, 0.33658442), (13, 0.109933175), (16, 0.3882941), (17, 0.098224804)]


위의 출력 결과에서 (숫자, 확률)은 각각 토픽 번호와 해당 토픽이 해당 문서에서 차지하는 분포도를 의미합니다.

위의 코드를 좀 더 깔끔한 형태인 데이터프레임 형식으로 출력해보겠습니다.

In [40]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [41]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,1.0,0.7064,"[(0, 0.12822452), (1, 0.7064364), (16, 0.14220..."
1,1,1.0,0.3948,"[(0, 0.2841266), (1, 0.39477527), (12, 0.17595..."
2,2,1.0,0.4828,"[(0, 0.2966694), (1, 0.4827575), (2, 0.0366607..."
3,3,12.0,0.4930,"[(6, 0.39444745), (12, 0.49300376)]"
4,4,16.0,0.3883,"[(7, 0.042748228), (12, 0.33659917), (13, 0.10..."
5,5,16.0,0.6884,"[(4, 0.05945181), (5, 0.13433759), (13, 0.0972..."
6,6,5.0,0.2687,"[(0, 0.10795893), (1, 0.26712227), (2, 0.05132..."
7,7,0.0,0.7820,"[(0, 0.7819813), (1, 0.08642423), (2, 0.061065..."
8,8,12.0,0.5867,"[(0, 0.25087175), (12, 0.58666867), (13, 0.091..."
9,9,0.0,0.4684,"[(0, 0.4684366), (6, 0.085227855), (7, 0.07909..."


## 6.3. LDA 실습2

앞서 gensim 을 통해 실습했지만, 이번에는 사이킷런을 통해 실습해보겠습니다.

### 6.3.1. 뉴스 기사 제목 데이터에 대한 이해

링크 : https://www.kaggle.com/therohk/million-headlines

In [42]:
import pandas as pd
import urllib.request

In [43]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv", filename="abcnews-date-text.csv")
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False)

In [44]:
print(len(data))

1082168


해당 데이터는 약 100만개의 샘플을 가지고 있으며, 상위 5개의 샘플만 출력해봅시다.

In [45]:
print(data.head(5))

   publish_date                                      headline_text
0      20030219  aba decides against community broadcasting lic...
1      20030219     act fire witnesses must be aware of defamation
2      20030219     a g calls for infrastructure protection summit
3      20030219           air nz staff in aust strike for pay rise
4      20030219      air nz strike to affect australian travellers


이 데이터는 publish_data와 headline_text라는 두 개의 열을 갖고 있습니다. 각각 뉴스가 나온 날짜와 뉴스 기사 제목을 의미합니다.

In [46]:
# 데이터는 날리고 뉴스 제목만 남깁니다.

text = data[['headline_text']]
text.head()

,headline_text
0,aba decides against community broadcasting lic...
1,act fire witnesses must be aware of defamation
2,a g calls for infrastructure protection summit
3,air nz staff in aust strike for pay rise
4,air nz strike to affect australian travellers


### 6.3.2. 텍스트 전처리

이번 챕터에서는 불용어 제거, 표제어 추출, 길이가 짧은 단어 제거 라는 세 가지 전처리 기법을 사용합니다.

In [47]:
import nltk

In [48]:
# 토큰화

text['headline_text'] = text.apply(lambda row: nltk.word_tokenize(row['headline_text']), axis=1)

C:\Users\thinp\anaconda3\envs\Wyatt37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [49]:
print(text.head(5))

                                       headline_text
0  [aba, decides, against, community, broadcastin...
1  [act, fire, witnesses, must, be, aware, of, de...
2  [a, g, calls, for, infrastructure, protection,...
3  [air, nz, staff, in, aust, strike, for, pay, r...
4  [air, nz, strike, to, affect, australian, trav...


In [50]:
from nltk.corpus import stopwords

In [51]:
# 불용어 처리

stop = stopwords.words('english')
text['headline_text'] = text['headline_text'].apply(lambda x: [word for word in x if word not in (stop)])

C:\Users\thinp\anaconda3\envs\Wyatt37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [52]:
print(text.head(5))

                                       headline_text
0   [aba, decides, community, broadcasting, licence]
1    [act, fire, witnesses, must, aware, defamation]
2     [g, calls, infrastructure, protection, summit]
3          [air, nz, staff, aust, strike, pay, rise]
4  [air, nz, strike, affect, australian, travellers]


In [53]:
# 표제어 추출 -> 3인칭 단수 표현을 1인칭으로 바꾸고, 과거형을 현재형으로 바꾼다.

from nltk.stem import WordNetLemmatizer

In [55]:
text['headline_text'] = text['headline_text'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])

C:\Users\thinp\anaconda3\envs\Wyatt37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [56]:
print(text.head())

                                       headline_text
0       [aba, decide, community, broadcast, licence]
1      [act, fire, witness, must, aware, defamation]
2      [g, call, infrastructure, protection, summit]
3          [air, nz, staff, aust, strike, pay, rise]
4  [air, nz, strike, affect, australian, travellers]


In [57]:
# 길이가 3 이하인 단어 삭제

tokenized_doc = text['headline_text'].apply(lambda x: [word for word in x if len(word) > 3])

In [58]:
print(tokenized_doc[:5])

0       [decide, community, broadcast, licence]
1      [fire, witness, must, aware, defamation]
2    [call, infrastructure, protection, summit]
3                   [staff, aust, strike, rise]
4      [strike, affect, australian, travellers]
Name: headline_text, dtype: object


### 6.3.3. TF-IDF 행렬 만들기

In [59]:
# 일단 TF-IDF 행렬을 만들기 위해 역토큰화를 시켜야 합니다.

detokenized_doc = []
for i in range(len(text)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)
    
text['headline_text'] = detokenized_doc # 다시 text['headline_text'] 에 재저장

C:\Users\thinp\anaconda3\envs\Wyatt37\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [60]:
text['headline_text'][:5]

0       decide community broadcast licence
1       fire witness must aware defamation
2    call infrastructure protection summit
3                   staff aust strike rise
4      strike affect australian travellers
Name: headline_text, dtype: object

In [61]:
# TF-IDF 행렬 만들기

from sklearn.feature_extraction.text import TfidfVectorizer

In [62]:
vectorizer = TfidfVectorizer(stop_words='english',
                            max_features=1000) # 상위 1,000개의 단어를 보존
X = vectorizer.fit_transform(text['headline_text'])
X.shape

(1082168, 1000)

### 6.3.4. 토픽 모델링

In [64]:
from sklearn.decomposition import LatentDirichletAllocation

In [65]:
lda_model = LatentDirichletAllocation(n_components=10,
                                     learning_method='online',
                                     random_state=42,
                                     max_iter=1)

lda_top = lda_model.fit_transform(X)

In [66]:
print(lda_model.components_)
print(lda_model.components_.shape)

[[1.00001578e-01 1.00001244e-01 3.50170827e+03 ... 1.00004257e-01
  1.00001722e-01 1.00003979e-01]
 [1.00001847e-01 1.00001158e-01 1.00002229e-01 ... 1.00005861e-01
  1.00006065e-01 1.00002605e-01]
 [1.00002307e-01 1.00001528e-01 1.00007348e-01 ... 1.00005637e-01
  1.00002505e-01 1.00005427e-01]
 ...
 [3.51600411e+02 1.00001057e-01 1.00003859e-01 ... 1.00003895e-01
  1.00005142e-01 1.00004596e-01]
 [1.00001062e-01 1.00001649e-01 1.00006522e-01 ... 1.77619511e+03
  1.50652738e+02 7.53381835e+02]
 [1.00004033e-01 1.13513398e+03 1.00030595e-01 ... 1.00009276e-01
  1.00003667e-01 1.00003605e-01]]
(10, 1000)


In [67]:
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d: " %(idx+1),
              [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n-1:-1]])
get_topics(lda_model.components_, terms)

Topic 1:  [('queensland', 7720.12), ('house', 6113.49), ('leave', 3849.71), ('2015', 3501.71), ('federal', 3120.93)]
Topic 2:  [('police', 12092.44), ('sydney', 8393.29), ('melbourne', 7528.43), ('south', 6677.03), ('open', 5663.0)]
Topic 3:  [('plan', 6033.16), ('change', 5874.27), ('year', 5586.42), ('school', 5465.06), ('coast', 5429.41)]
Topic 4:  [('kill', 5851.6), ('woman', 5456.76), ('help', 5225.56), ('minister', 3973.22), ('child', 3838.44)]
Topic 5:  [('death', 5935.06), ('women', 4232.53), ('indigenous', 4223.4), ('price', 3818.24), ('need', 3708.86)]
Topic 6:  [('interview', 5924.98), ('record', 3987.87), ('league', 3911.12), ('fight', 3872.94), ('dead', 3503.84)]
Topic 7:  [('charge', 8428.8), ('court', 7542.74), ('perth', 6456.53), ('murder', 6268.13), ('face', 5193.63)]
Topic 8:  [('trump', 11966.41), ('australian', 11088.95), ('election', 7561.63), ('adelaide', 6758.36), ('canberra', 6112.23)]
Topic 9:  [('home', 5674.38), ('market', 5545.86), ('north', 5142.38), ('warn